In [1]:
!nvidia-smi

Sun Aug 27 16:17:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.191.01   Driver Version: 450.191.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   27C    P0    58W / 300W |   5044MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
from datasets import load_dataset, Dataset, DatasetDict

wow = DatasetDict.load_from_disk("wow")
gen_mr = DatasetDict.load_from_disk("generated_data")

In [4]:
gen_mr

DatasetDict({
    train: Dataset({
        features: ['masked_response'],
        num_rows: 70188
    })
    valid: Dataset({
        features: ['masked_response'],
        num_rows: 3759
    })
    test: Dataset({
        features: ['masked_response'],
        num_rows: 3676
    })
})

In [5]:
wow

DatasetDict({
    train: Dataset({
        features: ['gold_pass', 'all_pass', 'gold_sen', 'all_sen', 'last_ut', 'context', 'response', 'context_eou', 'all_topic'],
        num_rows: 74092
    })
    valid: Dataset({
        features: ['gold_pass', 'all_pass', 'gold_sen', 'all_sen', 'last_ut', 'context', 'response', 'context_eou', 'all_topic'],
        num_rows: 3939
    })
    test: Dataset({
        features: ['gold_pass', 'all_pass', 'gold_sen', 'all_sen', 'last_ut', 'context', 'response', 'context_eou', 'all_topic'],
        num_rows: 3865
    })
})

In [6]:
def prep1(df):
    out = {"gold_pass": [], "last_ut": [], "all_pass": [], "all_topic": [], "all_sen": [], "response": [], "context_eou": []}
    for i, _ in enumerate(df["gold_pass"]):
        # print(type(k))
        if df["gold_pass"][i] == float("-inf"):
            continue
        for k in out.keys():
            out[k].append(df[k][i])
    return out
wow = wow.map(prep1, batched=True, remove_columns=wow["train"].column_names)

In [7]:
wow["train"][6557]["last_ut"]

'Do you like pizza?'

In [8]:
gen_mr["train"][6557]["masked_response"].replace("<pad>", "").replace("</s>", "")[1:]

"I love pizza! It's one of the most<extra_id_1><extra_id_2> in the<extra_id_0>."

In [9]:
import yake

language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

custom_kw_extractor_1 = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)


In [10]:
import spacy
def masker(inp, mask, ref_in, ref_out):
    op = inp
    msk = []
    j = 0
    for i, w in enumerate(mask):
        if w.lower() in ref_out.lower() and w.lower() not in ref_in.lower():
            op = op.replace(w, f"<extra_id_{j}>")
            j+=1
            msk.append(w)


    return op, " ".join(msk)

en = spacy.load('en_core_web_sm')
def key_exrt(k_lst, strng):

    keywords = []
    k2 = []
    if len(k_lst) == 0:
        keywords += [token for token in en(strng) if
              not token.is_punct
              and not token.is_currency
              and not token.is_digit
              and not token.is_oov
              and not token.is_space
              and not token.is_stop
              and not token.like_num
              and not token.pos_ in []]
    else:
        keywords += [key for key, _ in k_lst]
        k2 += [key for key, _ in k_lst]

    return keywords, ("; ".join(keywords) + ";"), k2

In [11]:
n = 1400000
ppp = [(1/2) - ((k**0.4)/(2*(n**0.4))) for k in range(n)]
ppp[int(n/2)]
a = ppp.pop()
len(ppp)

1399999

In [12]:
ppp[int(0)]

0.5

In [13]:
import random

# k = {}
# for i in range(10):
#     k[i] = 0
# for i in range(n*5):
#     k[int(random.random()*10)] += 1/(n*5)
# print(k)

In [14]:
# i
# print(i)
# i >= 1 

In [15]:
from tqdm.notebook import tqdm
sys_wow = {}
sys_wow["train"] = {"input": [], "pass_label": [], "output": []}
sys_wow["test"] = {"input": [], "pass_label": [], "output": []}
sys_wow["valid"] = {"input": [], "pass_label": [], "output": []}
n = 70188*3

for (var, fix), split in zip([(0, 1), (0, 0.5), (0, 1)], wow):
    for k, (i, (j, r)) in tqdm(enumerate((i, x) for _ in range(3) for i, x in enumerate(zip(wow[split], gen_mr[split])))):
        p = var*((k+1)**0.4)/(2*(n**0.4)) + fix
        c = random.random()
        c2 = random.random()
        query = ""
        dk = "?"

        keys = custom_kw_extractor_1.extract_keywords(j["response"])
        key, strng, js_ks = key_exrt(keys, j["response"])
        pas = j["all_pass"][int(j["gold_pass"])] if j["gold_pass"] != float("-inf") else ""
        jk, gpe_out = masker(j["response"], js_ks, j["context_eou"].replace(" <eou>", ""), pas)

        if c2 >= 0.03:
            if c <= p:
                query = r["masked_response"].replace("<pad>", "").replace("</s>", "")[1:]
                dk = "!"
            else:
                query = jk
                dk = ""


        input_q = j["last_ut"] + " <eou> " + query
        input_ids = []
        attention_mask = []
    #     out["pass_label"].append(int(df["gold_pass"][i]))
        for a, b in enumerate(j["all_pass"]):

            if a == int(j["gold_pass"]):
                # pass_label.append(1)
                b = (" ".join(j["all_sen"]))

            b = b.replace("no_passages_used", "")
            t = j["all_topic"][a]
            inp = f"question: {input_q} title: {t} passage: {b}"
            sys_wow[split]["pass_label"].append(int(j["gold_pass"]))
            sys_wow[split]["input"].append(inp)
            sys_wow[split]["output"].append(dk + gpe_out)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [16]:
from datasets import Dataset, DatasetDict
sys_dt = DatasetDict()
sys_dt["train"] = Dataset.from_dict(sys_wow["train"])
sys_dt["valid"] = Dataset.from_dict(sys_wow["valid"])
sys_dt["test"] = Dataset.from_dict(sys_wow["test"])

In [17]:
sys_dt

DatasetDict({
    train: Dataset({
        features: ['input', 'pass_label', 'output'],
        num_rows: 1473948
    })
    valid: Dataset({
        features: ['input', 'pass_label', 'output'],
        num_rows: 78939
    })
    test: Dataset({
        features: ['input', 'pass_label', 'output'],
        num_rows: 77196
    })
})

In [18]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
question = "question:"
title = "title:"
context = "passage:"
eou = "<eou>"
tokenizer.add_tokens([question, title, context, eou], special_tokens=True)

/raid/kavin-intern-maunendra/anaconda3/envs/wow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to rea

4

In [19]:
def tokenize_function(df):
    return tokenizer(df["input"], max_length=512, truncation=True, padding=False)
sys_dt = sys_dt.map(tokenize_function, batched=True)

Map:   0%|          | 0/1473948 [00:00<?, ? examples/s]

Map:   0%|          | 0/78939 [00:00<?, ? examples/s]

Map:   0%|          | 0/77196 [00:00<?, ? examples/s]

In [20]:
def tokenize_labels(df):
    out = tokenizer(df["output"], truncation=False, padding=False)
    return {"labels": out["input_ids"]}
sys_dt = sys_dt.map(tokenize_labels, batched=True, remove_columns=["output", "input"])

Map:   0%|          | 0/1473948 [00:00<?, ? examples/s]

Map:   0%|          | 0/78939 [00:00<?, ? examples/s]

Map:   0%|          | 0/77196 [00:00<?, ? examples/s]

In [21]:
sys_dt.save_to_disk("wow_gmr_100")

Saving the dataset (0/4 shards):   0%|          | 0/1473948 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/78939 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/77196 [00:00<?, ? examples/s]

In [32]:
sys_wow["train"]["output"][1333330]

"Kids' Choice Awards preview"

In [23]:
sys_dt 

DatasetDict({
    train: Dataset({
        features: ['pass_label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1473948
    })
    valid: Dataset({
        features: ['pass_label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 78939
    })
    test: Dataset({
        features: ['pass_label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 77196
    })
})

In [5]:
import torch
def prep1(df):
    out = {"pass_label": [], "attention_mask": [], "input_ids": []}
    for i, _ in enumerate(df["gold_pass"]):
        # print(type(k))
        if df["gold_pass"][i] == float("-inf"):
            continue
#         keys = custom_kw_extractor_1.extract_keywords(df["response"][i])
#         key, strng, js_ks = key_exrt(keys, df["response"][i])
#         pas = df["all_pass"][i][int(df["gold_pass"][i])] if df["gold_pass"][i] != float("-inf") else ""
#         msk_res, gpe_out = masker(df["response"][i], js_ks, df["context_eou"][i].replace(" <eou>", ""), pas)
        input_q = df["last_ut"][i] 
        input_ids = []
        attention_mask = []
#         out["pass_label"].append(int(df["gold_pass"][i]))
        for j, p in enumerate(df["all_pass"][i]):

            if j == int(df["gold_pass"][i]):
                # pass_label.append(1)
                p = (" ".join(df["all_sen"][i]))

            p = p.replace("no_passages_used", "")
            t = df["all_topic"][i][j]
            inp = tokenizer(f"question: {input_q} title: {t} passage: {p}", max_length=512, truncation=True, padding=False)
            out["input_ids"].append(inp["input_ids"])
            out["attention_mask"].append(inp["attention_mask"])
        
    #         le = 0
    #         for k in input_ids:
    #             if len(k) > le:
    #                 le = len(k)

    #         input_ids = [m + [tokenizer.pad_token_id]*(le-len(m)) for m in input_ids]
    #         attention_mask = [m + [tokenizer.pad_token_id]*(le-len(m)) for m in attention_mask]
            
            out["pass_label"].append(int(df["gold_pass"][i]))
#             out["labels"].append(tokenizer(gpe_out)["input_ids"])
#         out["input_ids"].append(torch.tensor(input_ids))
#         out["attention_mask"].append(torch.tensor(attention_mask))
        # out["masked_response"].append(input_q)
        # out["resp"].append(df["response"][i])

    return out

In [6]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers import T5Tokenizer
import torch
query_tokenizer = T5Tokenizer.from_pretrained("wow_qg_t5-large/final")
# query_tokenizer.decode(qg_wowd["test"]["labels"][175])
qg_model = T5ForConditionalGeneration.from_pretrained("wow_qg_t5-large/final")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [7]:
inp = torch.tensor(qg_wowd["train"]["input_ids"][17523]).view(1, -1)
att = torch.tensor(qg_wowd["train"]["attention_mask"][17523]).view(1, -1)
query_tokenizer.decode(qg_model.generate(input_ids=inp, attention_mask=att, num_beams=5, max_length=128).view(-1)[1:])

'Yep! Coors Brewing Company was founded in 1873 by two German immigrants, Adolph Coors and Jacob Schueler.</s>'

In [8]:
qg_wowd

DatasetDict({
    train: Dataset({
        features: ['gold_pass', 'all_pass', 'gold_sen', 'all_sen', 'last_ut', 'context', 'response', 'context_eou', 'all_topic', 'labels', 'attention_mask', 'input_ids'],
        num_rows: 70188
    })
    valid: Dataset({
        features: ['gold_pass', 'all_pass', 'gold_sen', 'all_sen', 'last_ut', 'context', 'response', 'context_eou', 'all_topic', 'labels', 'attention_mask', 'input_ids'],
        num_rows: 3759
    })
    test: Dataset({
        features: ['gold_pass', 'all_pass', 'gold_sen', 'all_sen', 'last_ut', 'context', 'response', 'context_eou', 'all_topic', 'labels', 'attention_mask', 'input_ids'],
        num_rows: 3676
    })
})

In [9]:
from torch.utils.data import DataLoader
data_collator = DataCollatorForSeq2Seq(query_tokenizer)
qg_wowd.set_format(type='torch', columns=['input_ids', 'attention_mask'])
# qg_wowd.set_format(type='torch', columns=qg_wowd["train"].column_names)
train_loader = DataLoader(qg_wowd["train"], 32, shuffle=False, collate_fn=data_collator)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
qg_model = qg_model.to(device)

In [25]:
from tqdm.notebook import tqdm
mdf_dt = {"train": {"masked_response": []}, "valid": {"masked_response": []}, "test": {"masked_response": []}}
qg_model.eval()
with torch.no_grad():
    for i in tqdm(train_loader):
        tokes = qg_model.generate(input_ids=i["input_ids"].to(device), attention_mask=i["attention_mask"].to(device), num_beams=5, max_length=128)
        texts = query_tokenizer.batch_decode(tokes, skip_special_tokens=True)
        mdf_dt["train"]["masked_response"] += texts

  0%|          | 0/2194 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB (GPU 0; 31.75 GiB total capacity; 23.95 GiB already allocated; 49.75 MiB free; 26.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [22]:
texts = query_tokenizer.batch_decode(tokes, skip_special_tokens=True)
texts[14]

'basically homebrewing is personal small scale beer production'

In [29]:
dt = Dataset.from_dict(mdf_dt)

In [31]:
dt.save_to_disk("123")

Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]

In [37]:
at = dt["masked_response"] + dt["masked_response"]*2

In [42]:
import random
random.choice([1, 3, 5, 7])

7